In [16]:
import argparse

import numpy as np
import cv2
import time
from sklearn.cluster import KMeans

NROT = 6
NPER = 8
NFILT = NROT*NPER
FILTSIZE = 49
NCLUSTERS = 4
TEXELSIZE = 4
pathName = "C:\\Users\\karan\\"
# fileName = "aerial-houses"
# fileName = "texture"
fileName = "grass-1"

def gaussian1d(sigma, mean, x, ord):
    x = np.array(x)
    x_ = x - mean
    var = sigma ** 2

    # Gaussian Function
    g1 = (1 / np.sqrt(2 * np.pi * var)) * (np.exp((-1 * x_ * x_) / (2 * var)))

    if ord == 0:
        g = g1
        return g
    elif ord == 1:
        g = -g1 * ((x_) / (var))
        return g
    else:
        g = g1 * (((x_ * x_) - var) / (var ** 2))
        return g


def gaussian2d(sup, scales):
    var = scales * scales
    shape = (sup, sup)
    n, m = [(i - 1) / 2 for i in shape]
    x, y = np.ogrid[-m:m + 1, -n:n + 1]
    g = (1 / np.sqrt(2 * np.pi * var)) * np.exp(-(x * x + y * y) / (2 * var))
    return g


def log2d(sup, scales):
    var = scales * scales
    shape = (sup, sup)
    n, m = [(i - 1) / 2 for i in shape]
    x, y = np.ogrid[-m:m + 1, -n:n + 1]
    g = (1 / np.sqrt(2 * np.pi * var)) * np.exp(-(x * x + y * y) / (2 * var))
    h = g * ((x * x + y * y) - var) / (var ** 2)
    return h


def makefilter(scale, phasex, phasey, pts, sup):
    gx = gaussian1d(3 * scale, 0, pts[0, ...], phasex)
    gy = gaussian1d(scale, 0, pts[1, ...], phasey)

    image = gx * gy

    image = np.reshape(image, (sup, sup))
    return image

def makeLMfilters():
    sup = 49
    scalex = np.sqrt(2) * np.array([1, 2, 3])
    norient = 6
    nrotinv = 12

    nbar = len(scalex) * norient
    nedge = len(scalex) * norient
    nf = nbar + nedge + nrotinv
    F = np.zeros([sup, sup, nf])
    hsup = (sup - 1) / 2

    x = [np.arange(-hsup, hsup + 1)]
    y = [np.arange(-hsup, hsup + 1)]

    [x, y] = np.meshgrid(x, y)

    orgpts = [x.flatten(), y.flatten()]
    orgpts = np.array(orgpts)

    count = 0
    for scale in range(len(scalex)):
        for orient in range(norient):
            angle = (np.pi * orient) / norient
            c = np.cos(angle)
            s = np.sin(angle)
            rotpts = [[c + 0, -s + 0], [s + 0, c + 0]]
            rotpts = np.array(rotpts)
            rotpts = np.dot(rotpts, orgpts)
            F[:, :, count] = makefilter(scalex[scale], 0, 1, rotpts, sup)
            F[:, :, count + nedge] = makefilter(scalex[scale], 0, 2, rotpts, sup)
            count = count + 1

    count = nbar + nedge
    scales = np.sqrt(2) * np.array([1, 2, 3, 4])

    for i in range(len(scales)):
        F[:, :, count] = gaussian2d(sup, scales[i])
        count = count + 1

    for i in range(len(scales)):
        F[:, :, count] = log2d(sup, scales[i])
        count = count + 1

    for i in range(len(scales)):
        F[:, :, count] = log2d(sup, 3 * scales[i])
        count = count + 1

    return F

def saveFilters(img):
    (height, width, depth) = img.shape
    count = 0
    for row in range(NPER):
        for col in range(NROT):
            tempImg = img[:, :, count]
            filename = "Filters\\LM_" + str(row) + "_" + str(col)
            normedFilter = normImg(tempImg)
            a = filename + ".png"
            cv2.imwrite(a, normedFilter)
            count = count + 1
    return

def applyLMFilters(img, filters):
    img = np.asarray(img)
    img = img.astype('float32')
    mean, std = img.mean(), img.std()
    img = (img - mean)/std
    row, col = img.shape
    responses = np.zeros((row,col,NFILT))
    for i in range(NFILT):
        b = filters[:,:,i]
        #b = normalize(b, axis = 1, norm = 'l1')
        res = cv2.filter2D(img, -1, b)
        #res = np.mean(res)
        responses[0:row, 0:col, i] = res
        #plt.imshow(res, cmap = 'gray')
        #plt.show()
    return responses

def normImg(img):
    tempImg = np.zeros_like(img)
    tempImg = (cv2.normalize(img, tempImg, 0.0, 127.0, cv2.NORM_MINMAX))
    res = (tempImg + 128.0).astype(np.uint8)
    return res

def makeMosaic(img):
    (height, width, depth) = img.shape
    res = np.zeros((height*8, width*6), np.float64)
    count = 0
    for row in range(8):
        for col in range(6):
            res[row*height:(row+1)*height, col*width:(col+1)*width] = \
            normImg(img[:, :, count])
            count = count + 1
    return res

def saveImage(img, name):
    cv2.imwrite(pathName + name + ".png", img)
    return

def formTexels(R, sz):
    h, w, c = R.shape
    l = int(h / sz)
    k = int(w / sz)
    i, j = 0, 0
    newR = np.zeros((h, w, NFILT))
    for z in range(NFILT):
        b = R[:, :, z]
        c = newR[:, :, z]
        for i in range(0, h, sz):
            if i + sz == h:
                i = h - sz
            for j in range(0, w, sz):
                if j + sz == w:
                    j = j - sz
                window = b[i:i + sz, j:j + sz]
                mean = np.mean(window)
                c[i:i + sz, j:j + sz] = mean
        newR[0:h, 0:w, z] = c
    return newR

# parser = argparse.ArgumentParser();
# parser.add_argument('--image_path', required=True, help='Absolute path of the image to be used.');
# if __name__ == '__main__':
#     args = parser.parse_args();
#     pathName = args.image_path;
#     print('IMAGE PATH: ', pathName);
currTime = time.time()
# Call the make filter function
F = makeLMfilters()
saveFilters(F)
saveImage(makeMosaic(F), "allFilters")
# load an image  
inputImage = cv2.cvtColor(cv2.imread(pathName + fileName + ".png"), cv2.COLOR_BGR2GRAY)

# find filter responses
rawR = applyLMFilters(inputImage, F)
R = formTexels(rawR, TEXELSIZE)

h, w, d = R.shape
R = np.reshape(R, (h * w, d))
# try segmenting
X = KMeans(n_clusters=NCLUSTERS, random_state=0).fit(R)
X = np.reshape(X.labels_, (h, w))
X = X / (NCLUSTERS-1)
X = 255 * X
X = np.array(X, dtype=np.uint8)
pcolor = cv2.applyColorMap(X, cv2.COLORMAP_JET)
saveImage(pcolor, fileName+"_Seg_"+str(NCLUSTERS))
elapsedTime = time.time() - currTime
print("Completed; elapsed time = ", elapsedTime)


Completed; elapsed time =  93.74730968475342
